In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from nets.cut import Cut
from pytorch_lightning import Trainer
# from data_loader_gaelle import ConcatDataset,ImageDataset,LotusDataModule, ConcatDataModule, LotusTrainTransforms2
import pandas as pd
import numpy as np 

from callbackss import logger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import NeptuneLogger
from pytorch_lightning.strategies.ddp import DDPStrategy

from monai.transforms import (  
    LoadImaged,
    Compose,
    Resize,
    RandZoomd,
    RandRotated,
    RandAffined,
    ToTensord
)   
import math

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
from PIL import Image
# import nrrd
import os
import sys
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import monai
from monai.transforms import (  
    LoadImaged,
    Compose,
    Resize,
    RandZoomd,
    RandRotated,
    RandAffined,
    ToTensord
)   
import math

import pytorch_lightning as pl


In [2]:
df_train_cbct = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/train.csv") 
df_val_cbct = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/valid.csv")  
df_test_cbct = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/test.csv")

df_train_mri = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/train.csv")
df_val_mri = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/valid.csv")   
df_test_mri = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/test.csv") 


print(df_train_mri.columns)  # Affiche les noms de colonnes du DataFrame
print(df_train_mri.head())   # Affiche les premières lignes pour vérifier les données


print("df_train_cbct : ",df_train_cbct)

Index(['index', 'img_fn', 'group', 'ignore_size_info', 'label'], dtype='object')
   index                  img_fn group  ignore_size_info  label
0      0  train/B040_cropped.nii     .                 0      1
1      1  train/B041_cropped.nii     .                 0      2
2      2  train/M001_cropped.nii     .                 0      3
3      3  train/M002_cropped.nii     .                 0      4
4      4  train/M003_cropped.nii     .                 0      5
df_train_cbct :      index                  img_fn group  ignore_size_info  label
0       0  train/B040_cropped.nii     .                 0      1
1       1  train/B041_cropped.nii     .                 0      2
2       2  train/M001_cropped.nii     .                 0      3
3       3  train/M002_cropped.nii     .                 0      4
4       4  train/M003_cropped.nii     .                 0      5
..    ...                     ...   ...               ...    ...
59     59  train/M061_cropped.nii     .                 0     6

In [3]:
transform = transforms.Compose(
    [   
        # EnsureChannelFirstd(keys=['img', 'seg'], channel_dim=-1),
        LoadImaged(keys=['img_mri', 'img_cbct']),
        RandZoomd(keys=['img_mri', 'img_cbct'], min_zoom=0.8, max_zoom=1.1, mode=['area', 'nearest'], prob=0.9, padding_mode='constant'),
        RandRotated(keys=['img_mri', 'img_cbct'], range_x=math.pi, mode=['bilinear', 'nearest'], prob=1.0),
        RandAffined(keys=['img_mri', 'img_cbct'], prob=0.8, shear_range=(0.1, 0.1), mode=['bilinear', 'nearest'], padding_mode='zeros'),
        ToTensord(keys=['img_mri', 'img_cbct'])
    ]

)

In [4]:
class LotusTrainTransforms2:
    def __init__(self, height: int = 256):

        # image augmentation functions
        self.train_transform = Compose(
            [   
            LoadImaged(keys=['img_mri', 'img_cbct']),
            RandZoomd(keys=['img_mri', 'img_cbct'], min_zoom=0.8, max_zoom=1.1, mode=['area', 'nearest'], prob=0.9, padding_mode='constant'),
            RandRotated(keys=['img_mri', 'img_cbct'], range_x=math.pi, mode=['bilinear', 'nearest'], prob=1.0),
            RandAffined(keys=['img_mri', 'img_cbct'], prob=0.8, shear_range=(0.1, 0.1), mode=['bilinear', 'nearest'], padding_mode='zeros'),
            ToTensord(keys=['img_mri', 'img_cbct'])
            ]
        )

    def __call__(self, inp):
        return self.train_transform(inp) 
   

class LotusDataset(Dataset):
    def __init__(self, df, mount_point = "./", img_column="img_path", seg_column=None):
        self.df = df
        self.mount_point = mount_point
        self.img_column = img_column
        self.seg_column = seg_column
        
        self.loader = LoadImaged(keys=["img_mri", "img_cbct"])

    def __len__(self):
        return len(self.df.index)

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.mount_point, self.df.iloc[idx][self.img_column])
        

        d = {"img_mri": img_path, "img_cbct": img_path}
        
        
        d = self.loader(d)
        
        return d
    
class LotusDataModule(pl.LightningDataModule):
    def __init__(self, df_train, df_val, df_test, mount_point="./", batch_size=256, num_workers=4, img_column="img_path", train_transform=None, valid_transform=None, test_transform=None, drop_last=False):
        super().__init__()

        self.df_train = df_train
        self.df_val = df_val
        self.df_test = df_test
        self.mount_point = mount_point
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.img_column = img_column        
        self.train_transform = train_transform
        self.valid_transform = valid_transform
        self.test_transform = test_transform
        self.drop_last=drop_last        

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        self.train_ds = monai.data.Dataset(data=LotusDataset(self.df_train, self.mount_point, img_column=self.img_column), transform=self.train_transform)
        self.val_ds = monai.data.Dataset(data=LotusDataset(self.df_val, self.mount_point, img_column=self.img_column), transform=self.valid_transform)
        self.test_ds = monai.data.Dataset(data=LotusDataset(self.df_test, self.mount_point, img_column=self.img_column), transform=self.test_transform)

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers=self.num_workers, persistent_workers=True, pin_memory=True, drop_last=self.drop_last, shuffle=True, prefetch_factor=2)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, num_workers=self.num_workers, persistent_workers=False, pin_memory=True, drop_last=self.drop_last)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, num_workers=self.num_workers, persistent_workers=True, pin_memory=True, drop_last=self.drop_last)
    
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets, use_max=True):
        self.datasets = datasets
        self.use_max = use_max

    def __getitem__(self, i):
        # print(f"Fetching index: {i}") #debug
        # for d in self.datasets:
            # print(f"Dataset length: {len(d)}") #debug
        return tuple(self.check_len(d, i) for d in self.datasets)

    def __len__(self):
        if self.use_max:
            return max(len(d) for d in self.datasets)
        return min(len(d) for d in self.datasets)

    def shuffle(self):
        for d in self.datasets:
            if isinstance(d, monai.data.Dataset):                
                d.data.df = d.data.df.sample(frac=1.0).reset_index(drop=True)                
            else:
                d.df = d.df.sample(frac=1.0).reset_index(drop=True)

    def check_len(self, d, i):
        if i < len(d):
            return d[i]
        else:
            j = i % len(d)
            return d[j]
    
class ConcatDataModule(pl.LightningDataModule):
    def __init__(self, datasetA_train, datasetA_val, datasetB_train, datasetB_val, batch_size=8, num_workers=4):
        super().__init__()

        self.datasetA_train = datasetA_train
        self.datasetB_train = datasetB_train

        self.datasetA_val = datasetA_val
        self.datasetB_val = datasetB_val
        
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders        
        self.train_ds =  ConcatDataset(self.datasetA_train, self.datasetB_train)
        self.val_ds = ConcatDataset(self.datasetA_val, self.datasetB_val)

    def train_dataloader(self):
        print("train_ds : ",self.train_ds)
        
        self.train_ds.shuffle()
        return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True)   

In [5]:
train_transform_mri = LotusTrainTransforms2()
valid_transform_mri = LotusTrainTransforms2()
MRI_data = LotusDataModule(df_train_mri, df_val_mri, df_test_mri, mount_point="/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_MRI/", batch_size=2, num_workers=4, img_column="img_fn", train_transform=train_transform_mri, valid_transform=valid_transform_mri, test_transform=valid_transform_mri, drop_last=False)

MRI_data.setup()

train_transform_cbct = LotusTrainTransforms2()
valid_transform_cbct = LotusTrainTransforms2()
CBCT_data = LotusDataModule(df_train_cbct, df_val_cbct, df_test_cbct, mount_point="/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/", batch_size=2, num_workers=4, img_column="img_fn", train_transform=train_transform_cbct, valid_transform=valid_transform_cbct, test_transform=valid_transform_cbct, drop_last=False)
CBCT_data.setup()

concat_data = ConcatDataModule(MRI_data.train_ds, MRI_data.val_ds, CBCT_data.train_ds, CBCT_data.val_ds, batch_size=2, num_workers=4)
concat_data.setup()
print("concat_data : ",concat_data)
print("oui")

concat_data :  <__main__.ConcatDataModule object at 0x7ef39c163700>
oui


In [7]:
print(torch.version.cuda)
for batch in concat_data.train_dataloader():
    images_mri, images_cbct = batch['img_mri'], batch['img_cbct']
    
    # Afficher les tailles des tensors
    print("9"*50)
    print("Images MRI:", images_mri.shape)
    print("Images CBCT:", images_cbct.shape)
    


12.1
train_ds :  <__main__.ConcatDataset object at 0x7ef27b2f3fd0>


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [1]:
import os
import torch

# Remplacez l'index par celui de votre GPU si nécessaire
gpu_index = 0

# Définir la variable d'environnement CUDA_VISIBLE_DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_index)
print(os.environ["CUDA_VISIBLE_DEVICES"])

# Vérifier si CUDA est disponible
cuda_available = torch.cuda.is_available()
print(torch.version.cuda)

if cuda_available:
    # Vérifier le nombre de GPU disponibles
    num_gpus = torch.cuda.device_count()
    print("num_gpus : ",num_gpus)

    # Afficher des informations sur chaque GPU disponible
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Afficher le GPU actuellement utilisé par PyTorch
    current_device = torch.cuda.current_device()
    print(f"GPU actuel: {torch.cuda.get_device_name(current_device)}")

    # Afficher la version de CUDA
    cuda_version = torch.version.cuda
    print(f"Version CUDA: {cuda_version}")
else:
    print("CUDA n'est pas disponible sur cet environnement.")

0
12.1
CUDA n'est pas disponible sur cet environnement.


/home/luciacev/APP/miniconda3/envs/pl/lib/python3.9/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
checkpoint_callback = ModelCheckpoint(
            dirpath='/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/output_train/',
            filename='{epoch}-{val_loss:.2f}',
            save_top_k=2,
            monitor='val_loss'
        )

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=30, verbose=True, mode="min")

callbacks=[early_stop_callback, checkpoint_callback]
neptune_logger = None

os.environ['NEPTUNE_API_TOKEN'] = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4ZDQ0NTI4Yi03ZWI3LTRiN2UtODAwMi04MThhYzAwNWJhZDgifQ=='

neptune_logger = NeptuneLogger(
    project='gaellel/MRICBCT',
    tags=["v1"],
    api_key=os.environ['NEPTUNE_API_TOKEN']
)

LOGGER = getattr(logger, "CutLogger")    
image_logger = LOGGER(log_steps=100)
callbacks.append(image_logger)


In [ ]:
trainer = Trainer(
    logger=neptune_logger,
    log_every_n_steps=100,
    max_epochs=200,
    max_steps=-1,
    callbacks=callbacks,
    accelerator='gpu', 
    devices=torch.cuda.device_count(),
    strategy=DDPStrategy(find_unused_parameters=False),
    reload_dataloaders_every_n_epochs=1
    # detect_anomaly=True
)
    

MisconfigurationException: `Trainer(strategy=<pytorch_lightning.strategies.ddp.DDPStrategy object at 0x79ff635400a0>)` is not compatible with an interactive environment. Run your code as a script, or choose a notebook-compatible strategy: `Trainer(strategy='ddp_notebook')`. In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.